In [67]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader


import numpy as np
import string
from tqdm import tqdm_notebook

In [68]:
all_letters = string.ascii_letters + " .,;'"
onGPU = False

In [69]:
def read_data():
    data_path = 'data/HPserie/HPSerie'
    with open(data_path, 'r') as myfile:
        data=myfile.read().replace('\n', '')
    return data

class TextDataset(Dataset):
    def __init__(self, data, seq_len, strided=True):
        self.data = data
        self.seq_len = seq_len
        self.strided = strided
        
    def __len__(self):
        if self.strided:
            return len(self.data) // (self.seq_len + 1)
        else:
            return len(self.data) - self.seq_len - 1
        
    def __getitem__(self, idx):
        step = self.seq_len + 1
        if self.strided:
            seq = self.data[idx*step:idx*step + step]
        else:
            seq = self.data[idx:idx+step]
        return {'X': np.array(seq[0:-1]),
                'Y': np.array(seq[1:])}
    
data = read_data()
data = [x for x in data if x in all_letters]

char_to_idx = {x: i for i,x in enumerate(all_letters)}
idx_to_char = {v: k for k,v in char_to_idx.items()}

data_enc = [char_to_idx[x] for x in data]
    
    
dataset = TextDataset(data_enc, 10)
dataloader = DataLoader(dataset, batch_size=5,
                        shuffle=True, num_workers=4)

In [70]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden, seq_len, batch_size):
        input = self.encoder(input)
        output, hidden = self.gru(input.view(seq_len, batch_size, -1), hidden)
        output = self.decoder(output.view(-1, self.hidden_size))
        output = output.view(seq_len, batch_size, self.output_size)
        return output, hidden

    def init_hidden(self, batch_size):
        if onGPU:
            return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size).cuda())
        else:
            return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))

In [71]:
input_size = len(all_letters)
hidden_size = 100
n_layers = 3

print_every = 50 
plot_every = 100
epochs = 10


model = RNN(input_size, hidden_size, input_size, n_layers)
model_optimizer = torch.optim.Adam(model.parameters())

if onGPU:
    model.cuda()

In [72]:
criterion = nn.CrossEntropyLoss()

def train(inp, target):
    hidden = model.init_hidden(inp.size()[1])
    model.zero_grad()
    loss = 0

    output, hidden = model(inp, hidden, inp.size()[0], inp.size()[1])
    output = output.view(-1, model.output_size)
    loss = criterion(output, target.contiguous().view(-1))

    loss.backward()
    model_optimizer.step()

    return loss.data[0]

In [ ]:
loss_avg = 0
losses = []

for epoch in range(1, epochs+1):
    bar = tqdm_notebook(dataloader, total=dataloader.__len__())
    for s in bar:
        X_enc, Y_enc = (s['X'].t(), s['Y'].t())
        if onGPU:
            loss = train(Variable(X_enc).cuda(), Variable(Y_enc).cuda())       
        else:
            loss = train(Variable(X_enc), Variable(Y_enc))       
        loss_avg += loss
        
        bar.set_postfix(loss=loss)
        if b == batch_per_epoch-1:
            bar.write(loss_avg / batch_per_epoch)
            loss_avg = 0